<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [62]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

In [63]:
for layer in resnet.layers:
    layer.trainable = False

In [64]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [20]:
# prep data 
from skimage.io import imread_collection
import os 

DATA_FOLDER = './data/'
M_FOLDER = DATA_FOLDER + 'mountain'
F_FOLDER = DATA_FOLDER + 'forest'

mountains = imread_collection([M_FOLDER+'/'+file for file in os.listdir(M_FOLDER)])
forests = imread_collection([F_FOLDER+'/'+file for file in os.listdir(F_FOLDER)])

In [21]:
len(mountains), len(forests)

(374, 328)

In [23]:
# what are the current size of our images?
mountains[0].shape, forests[0].shape

((256, 256, 3), (256, 256, 3))

In [32]:
# reshape each image to (224, 224) - size needed for resnet50
from skimage.transform import resize

def preprocess_image(image):
    return preprocess_input(resize(image, (224, 224)))

X = []
y = []

for i in range(len(mountains)):
    X.append(preprocess_image(mountains[i]))
    y.append(0)

for i in range(len(forests)):
    X.append(preprocess_image(forests[i]))
    y.append(1)

X = np.asarray(X)
y = np.asarray(y)

X.shape, y.shape

((702, 224, 224, 3), (702,))

In [48]:
# shuffle mountains and forests order 
'''
Another way:
idx = np.random.permutation(len(data))
x,y = data[idx], classes[idx]
'''

import random

c = list(zip(X, y))

random.shuffle(c)

X2, y2 = zip(*c)

X2 = np.asarray(X2)
y2 = np.asarray(y2)

X2.shape, y2.shape

((702, 224, 224, 3), (702,))

In [51]:
y2[:10]

array([0, 1, 1, 1, 1, 1, 0, 1, 1, 0])

In [60]:
from PIL import Image

img = Image.fromarray(preprocess_image(forests[0]), 'RGB')
img.show()

In [65]:
model.fit(X2, y2, epochs=10, validation_split=.2)

Train on 561 samples, validate on 141 samples
Epoch 1/10
561/561 [==============================] - 102s 181ms/sample - loss: 0.5281 - accuracy: 0.8164 - val_loss: 0.9869 - val_accuracy: 0.4397
Epoch 2/10
561/561 [==============================] - 91s 162ms/sample - loss: 0.1025 - accuracy: 0.9608 - val_loss: 1.2561 - val_accuracy: 0.4397
Epoch 3/10
561/561 [==============================] - 87s 154ms/sample - loss: 0.0916 - accuracy: 0.9643 - val_loss: 1.2834 - val_accuracy: 0.4397
Epoch 4/10
561/561 [==============================] - 90s 161ms/sample - loss: 0.0692 - accuracy: 0.9733 - val_loss: 1.0776 - val_accuracy: 0.4397
Epoch 5/10
561/561 [==============================] - 84s 149ms/sample - loss: 0.0353 - accuracy: 0.9857 - val_loss: 0.9596 - val_accuracy: 0.4397
Epoch 6/10
561/561 [==============================] - 71s 127ms/sample - loss: 0.0876 - accuracy: 0.9590 - val_loss: 1.3496 - val_accuracy: 0.4397
Epoch 7/10
561/561 [==============================] - 70s 124ms/sample 

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 109, 109, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 186624)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                11944000  
_________________________________________________________________
dense_13 (Dense)             (None, 1)                

In [45]:
# Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
# Fit Model
model.fit(X, y, validation_split=.2, epochs=10)

Train on 561 samples, validate on 141 samples
Epoch 1/10
561/561 [==============================] - 52s 93ms/sample - loss: 169.3359 - accuracy: 0.5152 - val_loss: 0.8979 - val_accuracy: 0.0000e+00
Epoch 2/10
561/561 [==============================] - 42s 75ms/sample - loss: 0.9845 - accuracy: 0.6631 - val_loss: 0.7015 - val_accuracy: 0.0000e+00
Epoch 3/10
561/561 [==============================] - 41s 74ms/sample - loss: 0.6970 - accuracy: 0.6631 - val_loss: 0.7120 - val_accuracy: 0.0000e+00
Epoch 4/10
561/561 [==============================] - 37s 66ms/sample - loss: 0.6850 - accuracy: 0.6667 - val_loss: 0.7275 - val_accuracy: 0.0000e+00
Epoch 5/10
561/561 [==============================] - 36s 65ms/sample - loss: 0.6802 - accuracy: 0.6667 - val_loss: 0.7437 - val_accuracy: 0.0000e+00
Epoch 6/10
561/561 [==============================] - 37s 66ms/sample - loss: 0.6756 - accuracy: 0.6667 - val_loss: 0.7627 - val_accuracy: 0.0000e+00
Epoch 7/10
561/561 [==============================] 

# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [ ]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language